# Matrix Product State (MPS) backend benchmark

This notebook runs a small selection of example circuits solely on the `Matrix Product State (MPS)` simulator. It demonstrates how to use `BenchmarkRunner` for collecting timing and memory metrics. Extend the `circuits_to_run` list or adjust parameters to analyze additional workloads.


In [ ]:
from benchmarks.backends import MPSAdapter
from benchmarks.runner import BenchmarkRunner
from benchmarks import circuits
import pandas as pd

In [ ]:
# Select representative circuits
circuits_to_run = [
    ("GHZ", circuits.ghz_circuit(5)),
    ("QFT", circuits.qft_circuit(5)),
    ("Grover", circuits.grover_circuit(5, 1)),
]

runner = BenchmarkRunner()
backend = MPSAdapter()
for name, circ in circuits_to_run:
    res = runner.run_multiple(circ, backend, return_state=False, repetitions=3)
    res["circuit"] = name

df = runner.dataframe()
df[["circuit", "prepare_time", "run_time", "total_time", "prepare_peak_memory", "run_peak_memory"]]


In [ ]:
import pandas as pd
from benchmarks.stats_utils import stats_table

def add_stats(df, quasar_col='QuASAr', baseline_cols=None, test='ttest', correction='bonferroni'):
    """Compute statistics comparing QuASAr with baselines.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame with per-circuit results. One column must correspond to QuASAr,
        others to baselines.
    quasar_col : str
        Name of the column containing QuASAr results.
    baseline_cols : list[str] | None
        Columns to treat as baselines. Defaults to all columns except quasar_col.
    test : str
        'ttest' or 'wilcoxon'.
    correction : str
        'bonferroni' or 'fdr_bh'.

    Returns
    -------
    pd.DataFrame
        Table with baseline name, statistic, corrected p-value, and effect size.
    """
    if baseline_cols is None:
        baseline_cols = [c for c in df.columns if c != quasar_col]
    baselines = {c: df[c] for c in baseline_cols}
    return stats_table(df[quasar_col], baselines, test=test, correction=correction)

# Example usage after computing results DataFrame named `results_df`:
# stats_df = add_stats(results_df)
# stats_df
